# Transformer Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
from pathlib import Path

wd = Path(os.path.dirname(os.path.abspath("__file__"))).parent.resolve()
sys.path.append(str(wd))

import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from datetime import datetime, timedelta
from src.models.auction_transformer import AuctionTransformer
from src.models.inference import predict_dataframe

pd.options.display.max_columns = None
pd.options.display.width = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [3]:
prediction_time = datetime.strptime("2025-04-12 00:00:00", "%Y-%m-%d %H:%M:%S")

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
        item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

feature_stats = torch.load('../generated/feature_stats.pt')

time_left_mapping = {
    'VERY_LONG': 48,
    'LONG': 12,
    'MEDIUM': 2,
    'SHORT': 0.5
}

In [ ]:
from src.data.utils import load_auctions_from_sample

data_dir = '../data/sample/'

df_auctions = load_auctions_from_sample(data_dir, prediction_time, time_left_mapping, item_to_idx, context_to_idx, bonus_to_idx, modtype_to_idx)

print("Auctions shape:", df_auctions.shape)
df_auctions.head()

In [ ]:
df_auctions[['current_hours', 'hours_on_sale']].describe()

In [ ]:
model = AuctionTransformer.load_from_checkpoint(
    '../models/auction_transformer_40M_39GB/last.ckpt',
    map_location=device
)

print(f'Number of model parameters: {sum(p.numel() for p in model.parameters())}')
model.eval()
print('Pre-trained Transformer model loaded successfully.')

In [ ]:
from src.models.inference import predict_dataframe

model = model.to('cuda')
df_auctions = predict_dataframe(model, df_auctions, prediction_time, feature_stats)

print("Mean hours on sale:", df_auctions['hours_on_sale'].mean())
print("Mean prediction:", df_auctions['prediction'].mean())
print("Mean sale probability:", df_auctions['sale_probability'].mean())

mae = mean_absolute_error(df_auctions['hours_on_sale'], df_auctions['prediction'])
print(f"Mean absolute error: {mae}")

df_auctions.head()

In [ ]:
df_auctions_filtered = df_auctions[df_auctions['current_hours'] <= 0]
df_auctions_filtered = df_auctions_filtered[df_auctions_filtered['time_left'] > 12.0]
#df_auctions_filtered = df_auctions_filtered[df_auctions_filtered['prediction'] <= 8.0]
len(df_auctions_filtered)

mae = mean_absolute_error(df_auctions_filtered['hours_on_sale'], df_auctions_filtered['prediction'])
print(f"Mean absolute error: {mae}") # 5.43

In [ ]:
df_auctions_filtered[['item_index', 'buyout','quantity', 'time_left', 'current_hours', 'hours_on_sale', 'prediction', 'sale_probability']].sample(10)

### Binary classification

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

df_auctions_filtered['sold_gt'] = df_auctions_filtered['hours_on_sale'] <= 8.0
df_auctions_filtered['sold_pred'] = df_auctions_filtered['sale_probability'] >= 0.75

accuracy = accuracy_score(df_auctions_filtered['sold_gt'], df_auctions_filtered['sold_pred'])
precision = precision_score(df_auctions_filtered['sold_gt'], df_auctions_filtered['sold_pred'])
recall = recall_score(df_auctions_filtered['sold_gt'], df_auctions_filtered['sold_pred'])
f1 = f1_score(df_auctions_filtered['sold_gt'], df_auctions_filtered['sold_pred'])

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

## Error analysis

In [ ]:
columns = [
    'item_index',
    'bid',
    'buyout',
    'quantity',
    'time_left',
    'first_appearance',
    'last_appearance',
    'current_hours',
    'hours_on_sale',
    'prediction',
    'sale_probability'
]

df_error = df_auctions_filtered[columns].copy()
df_error['error'] = np.abs(df_error['hours_on_sale'] - df_error['prediction'])

df_error.sample(5)

In [ ]:
df_error['time_left'].hist(bins=10)

In [ ]:
plt.boxplot(df_error[['hours_on_sale', 'prediction']])
plt.grid()
plt.show()

In [ ]:
# Define bins for hours_on_sale
bins = [(0,12), (12,24), (24,48)]

# Calculate mean error for each bin
for start, end in bins:
    mask = (df_error['hours_on_sale'] >= start) & (df_error['hours_on_sale'] <= end)
    mean_error = df_error[mask]['error'].mean()
    print(f"Mean error for hours {start}-{end}: {mean_error:.2f}")

# Create boxplot showing error distribution in each bin
error_by_bin = []
labels = []
for start, end in bins:
    mask = (df_error['hours_on_sale'] >= start) & (df_error['hours_on_sale'] <= end)
    error_by_bin.append(df_error[mask]['error'])
    labels.append(f"{start}-{end}h")

plt.boxplot(error_by_bin, labels=labels)
plt.title("Error Distribution by Hours on Sale")
plt.ylabel("Absolute Error")
plt.xlabel("Hours on Sale Range") 
plt.grid()
plt.show()

In [ ]:
# histogram of hours on sale and prediction
plt.hist(df_error['hours_on_sale'], bins=100, alpha=0.5, label='Hours on sale')
plt.hist(df_error['prediction'], bins=100, alpha=0.5, label='Prediction')
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
plt.hist(df_error['current_hours'], bins=15)
plt.grid()
plt.show()

We are interested in evaluating the model when the items are recently published, because this will be the main use case for the model

In [ ]:
query = (df_error['current_hours'] <= 12) & (df_error['time_left'] == 48.0)
query_df = df_error[query]
print(f"Mean sale probability: {query_df['sale_probability'].mean()}")
print(f"Mean error: {query_df['error'].mean()}")
print(f"Mean hours on sale: {query_df['hours_on_sale'].mean()}")
query_df.sample(10)

In [ ]:
query_df['hours_on_sale'].hist(bins=10)
plt.grid()
plt.show()

In [ ]:
corr_matrix = df_error[['bid', 'buyout', 'time_left', 'current_hours', 'sale_probability',
                        'hours_on_sale', 'prediction', 'error']].corr()

plt.figure(figsize=(10, 8))

sns.heatmap(corr_matrix, 
            annot=True, 
            cmap='coolwarm',
            vmin=-1, vmax=1, 
            center=0,
            fmt='.2f',
            square=True) 

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()

plt.show()

# Eval validation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import sys
from tqdm import tqdm
from datetime import datetime
pd.options.mode.chained_assignment = None

from pathlib import Path

repo_root = Path.cwd().parent.resolve()
sys.path.append(str(repo_root))

from sklearn.model_selection import train_test_split
from src.data.auction_dataset import AuctionDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
pairs = pd.read_csv('../generated/auction_indices.csv')
pairs = pairs[pairs['g_hours_on_sale_max'] < 50]
pairs = pairs[pairs['g_current_hours_max'] < 50]

train_pairs, val_pairs = train_test_split(pairs, test_size=0.05, random_state=42, shuffle=False)

print(f"Before filtering: {len(train_pairs)}")

train_pairs = train_pairs[train_pairs['g_hours_on_sale_len'] <= 64]
val_pairs = val_pairs[val_pairs['g_hours_on_sale_len'] <= 64]

print(f"After filtering: {len(train_pairs)}\n")

train_pairs = train_pairs[:int(len(train_pairs)*0.85)]

print(f"Train pairs: {len(train_pairs)}")
print(f"Val pairs: {len(val_pairs)}")

In [ ]:
val_pairs = val_pairs[val_pairs['record'] == '2025-04-01 00:00:00']
val_pairs.head()

In [8]:
import json
import os

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
    item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

feature_stats = torch.load('../generated/feature_stats.pt')

In [ ]:
val_pairs.tail()

In [ ]:
from src.models.auction_transformer import AuctionTransformer

model = AuctionTransformer.load_from_checkpoint(
    '../models/auction_transformer_40M_39GB/last.ckpt',
    map_location=device
)

print(f'Number of model parameters: {sum(p.numel() for p in model.parameters())}')
model.eval()
print('Pre-trained Transformer model loaded successfully.')

In [ ]:
from src.data.auction_dataset import AuctionDataset
from src.data.utils import collate_auctions

batch_size = 1024

val_dataset = AuctionDataset(val_pairs, feature_stats=feature_stats, path='../generated/sequences.h5')
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_auctions) 

In [ ]:
"""
validate_and_noise.py
─────────────────────
Assumes you already have:

* model            – your trained AuctionTransformer (on `device`)
* val_dataloader   – torch DataLoader created from AuctionDataset
* feature_stats    – dict with 'means' and 'stds'  (torch tensors 1-D)
"""

import itertools
from collections import defaultdict
import numpy as np
import torch
from tqdm import tqdm


# ───────────────────────────────────────────────────────────────────────────────
# 1. Helper: build a hashable signature of everything the network
#    can ACTUALLY distinguish once log1p+norm have been undone.
# ───────────────────────────────────────────────────────────────────────────────
def auction_signature(item_idx,
                      quantity,
                      buyout_gold,
                      context,
                      time_left,
                      current_hours,
                      bonus_lists,
                      modifier_types,
                      age_bucket=1.0):    # 1-hour buckets
    price_sig = round(float(buyout_gold), 2)          # 0.01 g precision
    age_sig   = int(current_hours // age_bucket)

    return (
        int(item_idx),
        int(quantity),
        price_sig,
        int(context),
        int(time_left),
        age_sig,
        tuple(sorted(bonus_lists)),
        tuple(sorted(modifier_types)),
    )


# ───────────────────────────────────────────────────────────────────────────────
# 2. Validation + noise-floor loop
# ───────────────────────────────────────────────────────────────────────────────
def evaluate_with_noise(model, val_loader, feature_stats, device="cuda"):
    means = feature_stats["means"].cpu().numpy()
    stds  = feature_stats["stds"].cpu().numpy()

    total_mse   = 0.0
    total_mae   = 0.0
    total_items = 0

    targets_by_sig = defaultdict(list)

    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            (auctions, item_index, contexts, bonus_lists,
             modifier_types, modifier_values, current_hours,
             time_left, buyout_ranking), y = batch

            # move to device ----------------------------------------------------
            auctions          = auctions.to(device)
            item_index        = item_index.to(device)
            contexts          = contexts.to(device)
            bonus_lists       = bonus_lists.to(device)
            modifier_types    = modifier_types.to(device)
            modifier_values   = modifier_values.to(device)
            current_hours     = current_hours.to(device)
            time_left         = time_left.to(device)
            buyout_ranking    = buyout_ranking.to(device)
            y                 = y.to(device)

            # forward -----------------------------------------------------------
            y_hat = model((auctions, item_index, contexts, bonus_lists,
                           modifier_types, modifier_values, buyout_ranking))

            # mask: real items, full 48 h left, listed ≤12 h ago -----------------
            mask  = (item_index != 0).float().unsqueeze(-1)
            #mask *= (time_left == 48.0).float().unsqueeze(-1)
            mask *= (current_hours <= 12.0).float().unsqueeze(-1)

            # losses ------------------------------------------------------------
            mse = torch.nn.functional.mse_loss(y_hat * mask,
                                               y.unsqueeze(2) * mask,
                                               reduction='sum')
            mae = torch.nn.functional.l1_loss(y_hat * mask * 48.0,
                                              y.unsqueeze(2) * mask * 48.0,
                                              reduction='sum')

            total_mse   += mse.item()
            total_mae   += mae.item()
            total_items += mask.sum().item()

            # ----- collect targets for irreducible-noise estimate --------------
            # work on CPU / numpy
            y_cpu            = (y * 48.0).cpu().numpy()
            mask_cpu         = mask.squeeze(-1).cpu().numpy()
            item_idx_cpu     = item_index.cpu().numpy()
            auctions_cpu     = auctions.cpu().numpy()        # z-scores
            contexts_cpu     = contexts.cpu().numpy()
            time_left_cpu    = time_left.cpu().numpy()
            current_hours_cpu= current_hours.cpu().numpy()
            bonus_lists_cpu  = bonus_lists.cpu().numpy()
            modifier_types_cpu = modifier_types.cpu().numpy()

            B, S, _ = auctions_cpu.shape
            for b in range(B):
                for s in range(S):
                    if not mask_cpu[b, s]:
                        continue

                    # -------- undo standardisation + log1p for price ------------
                    buyout_norm = auctions_cpu[b, s, 1]
                    log_buyout  = buyout_norm * stds[1] + means[1]
                    buyout_gold = np.expm1(log_buyout)

                    # quantity column is already linear but was standardised
                    qty_norm   = auctions_cpu[b, s, 2]
                    quantity   = int(round(qty_norm * stds[2] + means[2]))

                    sig = auction_signature(
                        item_idx_cpu[b, s],
                        quantity,
                        buyout_gold,
                        contexts_cpu[b, s],
                        time_left_cpu[b, s],
                        current_hours_cpu[b, s],
                        bonus_lists_cpu[b, s][bonus_lists_cpu[b, s] != 0],
                        modifier_types_cpu[b, s][modifier_types_cpu[b, s] != 0],
                    )

                    targets_by_sig[sig].append(float(y_cpu[b, s]))

    # ── aggregate --------------------------------------------------------------
    avg_mse = total_mse / total_items
    avg_mae = total_mae / total_items

    noise_sum   = 0.0
    noise_count = 0
    for t_list in targets_by_sig.values():
        if len(t_list) < 2:
            continue
        for a, b in itertools.combinations(t_list, 2):
            noise_sum   += abs(a - b)
            noise_count += 1

    irreducible_mae = noise_sum / noise_count if noise_count else 0.0

    print(f"Validation MSE          : {avg_mse:.4f}")
    print(f"Validation MAE          : {avg_mae:.4f} hours")
    print(f"Bayesian lower-bound MAE: {irreducible_mae:.4f} hours")

    return avg_mse, avg_mae, irreducible_mae

evaluate_with_noise(model, val_dataloader, feature_stats, device=device)

In [ ]:
from src.models.inference import predict_dataframe

predict_dataframe(model, df_auctions[df_auctions['item_index'] == 13815], prediction_time, feature_stats)